In [49]:
# Function calling definitions

old_functions = [
    {
        "name": "markdown_to_html",
        "description": "Convert Markdown text to HTML",
        "parameters": {
            "type": "object",
            "properties": {
                "markdown_text": {"type": "string", "description": "The Markdown text to convert"}
            },
            "required": ["markdown_text"]
        }
    },
    {
        "name": "call_llm_vision",
        "description": "Analyze an image using AI vision capabilities",
        "parameters": {
            "type": "object",
            "properties": {
                "image_path": {"type": "string", "description": "Path to the image file"},
                "prompt": {"type": "string", "description": "Prompt for image analysis"}
            },
            "required": ["image_path", "prompt"]
        }
    },
    {
        "name": "call_llm_general",
        "description": "Process a general text prompt using AI",
        "parameters": {
            "type": "object",
            "properties": {
                "prompt": {"type": "string", "description": "The text prompt to process"}
            },
            "required": ["prompt"]
        }
    },
    {
        "name": "call_llm_embeddings",
        "description": "Generate embeddings for a list of texts",
        "parameters": {
            "type": "object",
            "properties": {
                "texts": {"type": "array", "items": {"type": "string"}, "description": "List of texts to generate embeddings for"}
            },
            "required": ["texts"]
        }
    },
    {
        "name": "execute_sql_query",
        "description": "Execute a SQL query on a SQLite database",
        "parameters": {
            "type": "object",
            "properties": {
                "db_path": {"type": "string", "description": "Path to the SQLite database file"},
                "query": {"type": "string", "description": "SQL query to execute"}
            },
            "required": ["db_path", "query"]
        }
    },
    {
        "name": "run_shell_commands",
        "description": "Run a list of shell commands",
        "parameters": {
            "type": "object",
            "properties": {
                "commands": {"type": "array", "items": {"type": "string"}, "description": "List of shell commands to execute"}
            },
            "required": ["commands"]
        }
    },
        {
        "name": "count_wednesdays",
        "type": "function",
        "function": {
            "name": "count_wednesdays",
            "description": "Counts the number of Wednesdays in a list of dates from an input file and writes the count to an output file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "input_file": {
                        "type": "string",
                        "description": "Path to the input file containing dates."
                    },
                    "output_file": {
                        "type": "string",
                        "description": "Path to the output file where the count will be written."
                    }
                },
                "required": ["input_file", "output_file"]
            }
        }
    }
]


In [5]:
from openai import OpenAI
from pydantic import BaseModel
from openai import OpenAI
import os
from dotenv import load_dotenv
from pathlib import Path
import subprocess
import json
from typing import Dict, Any, Optional

In [6]:
load_dotenv(".env")

AIPROXY_TOKEN = os.getenv("AIPROXY_TOKEN")
API_BASE_URL = "https://aiproxy.sanand.workers.dev/openai/v1"

client = OpenAI(
    api_key=AIPROXY_TOKEN,
    base_url=API_BASE_URL
)

In [62]:
tools = [
    {
        "name": "count_wednesdays",
        "description": "Counts the number of Wednesdays in a list of dates and writes the count to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "input_file": {"type": "string", "description": "Path to the file containing a list of dates."},
                "output_file": {"type": "string", "description": "Path to the file to write the count of Wednesdays."}
            },
            "required": ["input_file", "output_file"]
        }
    },
    {
        "name": "sort_contacts",
        "description": "Sorts a JSON array of contacts by last_name and first_name and writes to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "input_file": {"type": "string", "description": "Path to the JSON file containing contacts."},
                "output_file": {"type": "string", "description": "Path to the JSON file to write the sorted contacts."}
            },
            "required": ["input_file", "output_file"]
        }
    },
    {
        "name": "write_recent_logs",
        "description": "Extracts the first line of the 10 most recent .log files in a directory and writes them to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory": {"type": "string", "description": "Path to the directory containing .log files."},
                "output_file": {"type": "string", "description": "Path to the file to write the recent logs."}
            },
            "required": ["directory", "output_file"]
        }
    },
    {
        "name": "generate_markdown_index",
        "description": "Creates an index mapping Markdown file names to their titles from H1 headings.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory": {"type": "string", "description": "Path to the directory containing Markdown files."},
                "output_file": {"type": "string", "description": "Path to the JSON file to write the index."}
            },
            "required": ["directory", "output_file"]
        }
    },
    {
        "name": "extract_email_sender",
        "description": "Extracts the sender's email address from an email file and writes it to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "input_file": {"type": "string", "description": "Path to the file containing the email message."},
                "output_file": {"type": "string", "description": "Path to the file to write the extracted email address."}
            },
            "required": ["input_file", "output_file"]
        }
    },
    {
        "name": "extract_credit_card_number",
        "description": "Extracts the credit card number from an image and writes it without spaces to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "input_image": {"type": "string", "description": "Path to the image file containing the credit card number."},
                "output_file": {"type": "string", "description": "Path to the file to write the extracted credit card number."}
            },
            "required": ["input_image", "output_file"]
        }
    },
    {
        "name": "find_similar_comments",
        "description": "Finds the most similar pair of comments using embeddings and writes them to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "input_file": {"type": "string", "description": "Path to the file containing comments."},
                "output_file": {"type": "string", "description": "Path to the file to write the most similar pair of comments."}
            },
            "required": ["input_file", "output_file"]
        }
    },
    {
        "name": "calculate_gold_ticket_sales",
        "description": "Calculates the total sales of 'Gold' ticket type and writes the result to a file.",
        "parameters": {
            "type": "object",
            "properties": {
                "database_file": {"type": "string", "description": "Path to the SQLite database file."},
                "output_file": {"type": "string", "description": "Path to the file to write the total sales."}
            },
            "required": ["database_file", "output_file"]
        }
    }
]



In [79]:
from openai import OpenAI, pydantic_function_tool
from pydantic import BaseModel, Field

# Define Pydantic models for input parameters
class CountWednesdaysInput(BaseModel):
    input_file: str = Field(...,description="Path to the file containing a list of dates.")
    output_file: str = Field(..., description="Path to the file to write the count of Wednesdays.")


class SortContactsInput(BaseModel):
    input_file: str = Field(..., description="Path to the JSON file containing contacts.")
    output_file: str = Field(..., description="Path to the JSON file to write the sorted contacts.")


class WriteRecentLogsInput(BaseModel):
    directory: str = Field(..., description="Path to the directory containing .log files.")
    output_file: str = Field(..., description="Path to the file to write the recent logs.")


class GenerateMarkdownIndexInput(BaseModel):
    directory: str = Field(..., description="Path to the directory containing Markdown files.")
    output_file: str = Field(..., description="Path to the JSON file to write the index.")


class ExtractEmailSenderInput(BaseModel):
    input_file: str = Field(..., description="Path to the file containing the email message.")
    output_file: str = Field(..., description="Path to the file to write the extracted email address.")


class ExtractCreditCardNumberInput(BaseModel):
    input_image: str = Field(..., description="Path to the image file containing the credit card number.")
    output_file: str = Field(..., description="Path to the file to write the extracted credit card number.")


class FindSimilarCommentsInput(BaseModel):
    input_file: str = Field(..., description="Path to the file containing comments.")
    output_file: str = Field(..., description="Path to the file to write the most similar pair of comments.")


class Execute_SQL_Query(BaseModel):
    database_file: str = Field(..., description="Path to the SQLite database file.")
    query: str = Field(..., description="The sql query to execute")
    output_file: str = Field(..., description="Path to the file to write the total sales.")


In [80]:
# Define tools using pydantic_function_tool
pydantic_tools = [
    pydantic_function_tool(CountWednesdaysInput),
    pydantic_function_tool(SortContactsInput),
    pydantic_function_tool(WriteRecentLogsInput),
    pydantic_function_tool(GenerateMarkdownIndexInput),
    pydantic_function_tool(ExtractEmailSenderInput),
    pydantic_function_tool(ExtractCreditCardNumberInput),
    pydantic_function_tool(FindSimilarCommentsInput),
    pydantic_function_tool(Execute_SQL_Query),
]


In [75]:
task = """
 The SQLite database file /data/ticket-sales.db has a tickets with columns type, units, and price. Each row is a customer bid for a concert ticket. What is the total sales of all the items in the “Gold” ticket type? Write the number in /data/ticket-sales-gold.txt
"""

In [76]:
email="23f2001413@ds.study.iitm.ac.in"


system_prompt = """ 
You are an AI assistant that helps with various tasks. Use the provided functions to complete the user's request.

Guidelines:
1. Always check whether a tool or dependency is installed before running any command. If it is missing, install it first using the appropriate package manager.
2. Remote files (e.g., from URLs) must be downloaded first before they can be used. If a task involves executing a remote script, download the script to a file in the `/data` directory, and then run it.
3. Ensure commands are safe and do not exfiltrate data outside `/data`. You must never delete files or data outside `/data`.
4. Return only valid shell commands in JSON format with a 'command' key.
   - If a task is invalid or cannot be completed, return an empty command string in JSON format.
5. Always verify ambiguous terms in the task description.
   - For example, ensure 'uv' refers to the package manager and not `uvicorn`.
6. Tasks may be in different languages. Always interpret them correctly.
7. Ensure commands are well-formed, functional, and execute in sequence as required for the task.

"""
    

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": task}
]

In [81]:


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=pydantic_tools,
    function_call="auto"
)

response_message = response.choices[0].message
    


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'functions[0].name'.", 'type': 'invalid_request_error', 'param': 'functions[0].name', 'code': 'missing_required_parameter'}, 'monthlyCost': 0.063069, 'cost': 0, 'monthlyRequests': 98}

In [73]:

if response_message.function_call:
    function_name = response_message.function_call.name
    function_args = json.loads(response_message.function_call.arguments)
    
    print(function_name) 
    print(function_args)
    
    

write_recent_logs
{'directory': '/data/logs/', 'output_file': '/data/logs-recent.txt'}


In [74]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"directory":"/data/logs/","output_file":"/data/logs-recent.txt"}', name='write_recent_logs'), tool_calls=None)